## NewsFlow ETL

### Import Libraries and Connect to the News API

In [ ]:
import pandas as pd
import sqlite3
from newsapi import NewsApiClient
import logging
from config import api_key

In [ ]:
news_api = NewsApiClient(api_key=api_key['API_KEY'])

### Retrieve and Print News Articles

In [ ]:
def extract_news_data():
    try:
        result = news_api.get_everything(q="AI", language="en",sort_by='publishedAt')
        logging.info("Connection is successful.")
        return result["articles"]
    except:
        logging.error("Connection is unsuccessful.")
        return None

articles = extract_news_data()

print(articles[:3])

### Clean Author Column

### Transform News Data

### Load the Data into SQLite Database


### Verify Data Loading